In [ ]:
!pip install sovai[full]

## Pairwise Interactions

### Interactions

1. Cross-Sectional - The cross-sectional information between two tickers are compared.
   1. Mean, Std, Skew, First Diff, Zero Crossing, Turning Points
   3. Spectral Centroid, Hjorst, Hurst, Time Reversal
2. Time-Series - The time-series information between two tickers are compared.
   1. DTW (Dynamic Time Warping)
   2. Pearson Correlation. 
3. Panel Interactions - The entire dataset are compared on different axis.
   1. Tucker Decomposition


In [26]:
import sovai as sov

sov.token_auth(token="visit https://sov.ai/profile for your token")

In [31]:
# Load ratios - takes around 
df_factors = sov.data("factors/accounting", start_date="2020-01-26", purge_cache=True); df_factors.head()

Loading partitions: 100%|██████████| 1936/1936 [00:27<00:00, 69.77partition/s]


profitability  value  solvency  cash_flow  illiquidity  \
ticker date                                                                 
A      2020-01-31         67.000 10.000    51.000     77.000       64.000   
       2020-02-07         67.000 10.000    51.000     77.000       84.000   
       2020-02-14         67.000 10.000    51.000     77.000       68.000   
       2020-02-21         68.000 10.000    51.000     77.000       90.000   
       2020-02-28         68.000 10.000    51.000     76.000       89.000   

                   momentum_long_term  momentum_medium_term  \
ticker date                                                   
A      2020-01-31              80.000                84.000   
       2020-02-07              83.000                83.000   
       2020-02-14              83.000                80.000   
       2020-02-21              85.000                76.000   
       2020-02-28              82.000                63.000   

                   short_term_reversal  price_volatility  dividend_yield  \
ticker date                                                                
A      2020-01-31               74.000            33.000          33.000   
       2020-02-07               73.000            30.000          30.000   
       2020-02-14               39.000            29.000          29.000   
       2020-02-21               49.000            28.000          28.000   
       2020-02-28               46.000            23.000          23.000   

                   earnings_consistency  small_size  low_growth  \
ticker date                                                       
A      2020-01-31                94.000       8.000      52.000   
       2020-02-07                88.000       8.000      52.000   
       2020-02-14                94.000       8.000      52.000   
       2020-02-21                85.000       8.000      52.000   
       2020-02-28                88.000       8.000      52.000   

                   low_equity_issuance  bounce_dip  accrual_growth  \
ticker date                                                          
A      2020-01-31               97.000      27.000          83.000   
       2020-02-07               97.000      29.000          83.000   
       2020-02-14               97.000      29.000          82.000   
       2020-02-21               97.000      31.000          80.000   
       2020-02-28               96.000      30.000          78.000   

                   low_depreciation_growth  current_liquidity  low_rnd  \
ticker date                                                              
A      2020-01-31                   65.000             80.000   26.000   
       2020-02-07                   65.000             80.000   26.000   
       2020-02-14                   65.000             80.000   26.000   
       2020-02-21                   65.000             80.000   27.000   
       2020-02-28                   66.000             80.000   27.000   

                   momentum  
ticker date                  
A      2020-01-31    86.000  
       2020-02-07    88.000  
       2020-02-14    87.000  
       2020-02-21    85.000  
       2020-02-28    78.000

In [32]:
df_factors.query("ticker == 'AAPL'").align(df_factors.query("ticker == 'AMZN'"), join='inner', axis=0)

(Empty CustomDataFrame
 Columns: [profitability, value, solvency, cash_flow, illiquidity, momentum_long_term, momentum_medium_term, short_term_reversal, price_volatility, dividend_yield, earnings_consistency, small_size, low_growth, low_equity_issuance, bounce_dip, accrual_growth, low_depreciation_growth, current_liquidity, low_rnd, momentum]
 Index: [],
 Empty CustomDataFrame
 Columns: [profitability, value, solvency, cash_flow, illiquidity, momentum_long_term, momentum_medium_term, short_term_reversal, price_volatility, dividend_yield, earnings_consistency, small_size, low_growth, low_equity_issuance, bounce_dip, accrual_growth, low_depreciation_growth, current_liquidity, low_rnd, momentum]
 Index: [])

In [33]:
df_slice = df_factors.select_stocks("mega").date_range("2020-01-01")

### Standard Cross-Sectional Distance (Mean, Cosine)

In [8]:
dist_matrix = df_slice.distance()

In [9]:
dist_matrix

,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMZN,ASML,AVGO,AXP,AZN,BA,BABA,BAC,BHP,BP,BRK.B,BUD,C,CAT,CHL,CMCSA,COST,CRM,CSCO,CVX,DHR,DIS,GE,GOOGL,HD,HSBC,IBM,INTC,ISRG,JNJ,JPM,KO,LIN,LLY,MA,MCD,META,MRK,MS,MSFT,NFLX,NKE,NOW,NVDA,NVO,NVS,ORCL,PDD,PEP,PFE,PG,PLTR,PM,PTRCY,PYPL,QCOM,SAP,SHEL,T,TM,TMO,TMUS,TSLA,TSM,TTE,UNH,V,VALE,VZ,WFC,WMT,XOM
AAPL,0.000,0.073,0.023,0.104,0.059,0.310,0.291,0.174,0.116,0.090,0.100,0.079,0.640,0.146,0.202,0.096,0.221,0.154,0.192,0.324,0.055,0.261,0.152,0.110,0.164,0.032,0.114,0.040,0.158,0.234,0.139,0.104,0.160,0.075,0.182,0.301,0.029,0.145,0.080,0.038,0.037,0.112,0.110,0.209,0.033,0.189,0.082,0.191,0.116,0.248,0.320,0.080,0.050,0.067,0.341,0.113,0.089,0.094,0.611,0.169,0.256,0.174,0.120,0.054,0.301,0.253,0.242,0.054,0.152,0.467,0.168,0.178,0.143,0.136,0.280,0.227,0.312,0.127,0.156
ABBV,0.073,0.000,0.094,0.164,0.111,0.388,0.271,0.107,0.174,0.111,0.204,0.034,0.660,0.163,0.298,0.224,0.232,0.228,0.215,0.390,0.129,0.267,0.173,0.170,0.191,0.111,0.210,0.075,0.192,0.330,0.237,0.123,0.162,0.071,0.184,0.372,0.084,0.228,0.153,0.151,0.064,0.166,0.158,0.273,0.063,0.222,0.121,0.236,0.267,0.222,0.381,0.111,0.116,0.061,0.339,0.149,0.136,0.208,0.643,0.201,0.269,0.286,0.183,0.136,0.318,0.307,0.278,0.099,0.153,0.456,0.172,0.217,0.194,0.266,0.316,0.264,0.454,0.208,0.243
ABT,0.023,0.094,0.000,0.130,0.063,0.305,0.309,0.162,0.110,0.117,0.158,0.096,0.643,0.135,0.231,0.117,0.212,0.188,0.196,0.362,0.099,0.290,0.189,0.164,0.201,0.043,0.128,0.055,0.197,0.243,0.145,0.165,0.203,0.088,0.191,0.300,0.029,0.172,0.107,0.054,0.069,0.162,0.142,0.241,0.040,0.226,0.104,0.262,0.167,0.319,0.394,0.123,0.060,0.105,0.372,0.179,0.113,0.153,0.647,0.223,0.247,0.206,0.172,0.063,0.370,0.289,0.276,0.092,0.223,0.561,0.199,0.197,0.221,0.205,0.364,0.299,0.361,0.194,0.204
ACN,0.104,0.164,0.130,0.000,0.108,0.353,0.325,0.211,0.108,0.218,0.124,0.243,0.799,0.234,0.243,0.126,0.342,0.133,0.238,0.357,0.220,0.398,0.194,0.079,0.255,0.198,0.240,0.114,0.192,0.444,0.183,0.079,0.270,0.280,0.333,0.267,0.180,0.158,0.119,0.157,0.152,0.051,0.107,0.237,0.192,0.162,0.100,0.297,0.145,0.352,0.375,0.149,0.248,0.216,0.385,0.141,0.272,0.168,0.697,0.177,0.379,0.318,0.302,0.203,0.442,0.338,0.239,0.144,0.177,0.544,0.229,0.290,0.143,0.134,0.444,0.293,0.430,0.180,0.286
ADBE,0.059,0.111,0.063,0.108,0.000,0.461,0.161,0.087,0.049,0.115,0.194,0.125,0.721,0.074,0.329,0.184,0.310,0.257,0.310,0.443,0.162,0.341,0.267,0.174,0.111,0.078,0.178,0.040,0.258,0.401,0.061,0.179,0.243,0.179,0.158,0.153,0.073,0.271,0.215,0.140,0.068,0.127,0.202,0.079,0.068,0.278,0.038,0.186,0.159,0.190,0.221,0.049,0.130,0.114,0.229,0.232,0.138,0.238,0.459,0.271,0.322,0.132,0.118,0.090,0.382,0.422,0.357,0.080,0.281,0.400,0.099,0.261,0.221,0.174,0.347,0.366,0.467,0.271,0.224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VALE,0.280,0.316,0.364,0.444,0.347,0.315,0.420,0.459,0.432,0.311,0.267,0.282,0.299,0.245,0.275,0.308,0.174,0.259,0.325,0.210,0.178,0.110,0.262,0.331,0.308,0.235,0.133,0.281,0.232,0.230,0.398,0.276,0.084,0.247,0.128,0.584,0.288,0.311,0.391,0.334,0.267,0.406,0.270,0.384,0.265,0.317,0.374,0.264,0.251,0.336,0.411,0.308,0.258,0.185,0.353,0.302,0.157,0.316,0.502,0.222,0.194,0.218,0.186,0.277,0.022,0.220,0.260,0.243,0.308,0.408,0.380,0.120,0.234,0.333,0.000,0.219,0.249,0.239,0.082
VZ,0.227,0.264,0.299,0.293,0.366,0.250,0.656,0.494,0.473,0.307,0.165,0.268,0.456,0.372,0.162,0.160,0.229,0.153,0.124,0.157,0.136,0.196,0.041,0.192,0.443,0.254,0.182,0.280,0.101,0.256,0.489,0.097,0.205,0.209,0.295,0.771,0.275,0.154,0.135,0.244,0.308,0.221,0.120,0.557,0.271,0.190,0.362,0.305,0.188,0.471,0.584,0.352,0.253,0.197,0.601,0.081,0.201,0.111,0.884,0.083,0.212,0.374,0.286,0.341,0.218,0.058,0.087,0.203,0.085,0.564,0.404,0.124,0.107,0.223,0.219,0.000,0.207,0.088,0.155

In [10]:
dist_matrix.sort_values("AAPL")[["AAPL"]].T

,AAPL,ABT,JNJ,CSCO,MRK,LLY,LIN,DHR,NVS,SAP,TMO,CAT,ADBE,ORCL,ABBV,IBM,AZN,NVO,KO,MSFT,PFE,AVGO,PG,BHP,AXP,ACN,HD,MCD,COST,MA,PEP,CVX,ASML,NKE,QCOM,WMT,V,GOOGL,UNH,JPM,BABA,TMUS,CMCSA,BRK.B,XOM,DIS,HSBC,CRM,TSM,PM,PYPL,AMZN,TTE,INTC,MS,NFLX,BUD,BAC,META,BP,VZ,GE,TM,NOW,T,PTRCY,CHL,VALE,AMD,ISRG,SHEL,AIG,WFC,NVDA,C,PDD,TSLA,PLTR,BA
AAPL,0.000,0.023,0.029,0.032,0.033,0.037,0.038,0.040,0.050,0.054,0.054,0.055,0.059,0.067,0.073,0.075,0.079,0.080,0.080,0.082,0.089,0.090,0.094,0.096,0.100,0.104,0.104,0.110,0.110,0.112,0.113,0.114,0.116,0.116,0.120,0.127,0.136,0.139,0.143,0.145,0.146,0.152,0.152,0.154,0.156,0.158,0.160,0.164,0.168,0.169,0.174,0.174,0.178,0.182,0.189,0.191,0.192,0.202,0.209,0.221,0.227,0.234,0.242,0.248,0.253,0.256,0.261,0.280,0.291,0.301,0.301,0.310,0.312,0.320,0.324,0.341,0.467,0.611,0.640


#### Other Cross-Sectional Calculations

The extracted features include `mean`, `ske`, `std` (standard deviation), `diffm` (first difference mean), `zcr` (zero crossing rate), `mac` (mean absolute change), `sc` (spectral centroid), `tp` (turning points), `acl1` (autocorrelation at lag 1), `hjorthm` (Hjorth mobility), `hurst` (Hurst exponent), `hist` (histogram mode with 5 bins), and `timerev` (time reversibility statistic). These features capture various aspects of the time series, including central tendency, variability, complexity, and temporal characteristics.

In [11]:
features = ['mean', 'skew', 'std', 'diffm', 'zcr', 'mac', 'sc', 'tp', 'acl1', 'hjorthm', 'hurst', 'hist', 'timerev']

dist_matrix_all = df_slice.distance(orient="cross-sectional", on='ticker', distance='cosine', calculations=features)

In [12]:
dist_matrix_all

,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMZN,ASML,AVGO,AXP,AZN,BA,BABA,BAC,BHP,BP,BRK.B,BUD,C,CAT,CHL,CMCSA,COST,CRM,CSCO,CVX,DHR,DIS,GE,GOOGL,HD,HSBC,IBM,INTC,ISRG,JNJ,JPM,KO,LIN,LLY,MA,MCD,META,MRK,MS,MSFT,NFLX,NKE,NOW,NVDA,NVO,NVS,ORCL,PDD,PEP,PFE,PG,PLTR,PM,PTRCY,PYPL,QCOM,SAP,SHEL,T,TM,TMO,TMUS,TSLA,TSM,TTE,UNH,V,VALE,VZ,WFC,WMT,XOM
AAPL,0.000,0.129,0.064,0.134,0.077,0.262,0.257,0.143,0.092,0.081,0.112,0.093,0.640,0.326,0.193,0.117,0.230,0.152,0.236,0.301,0.040,0.317,0.158,0.111,0.186,0.067,0.210,0.069,0.245,0.260,0.104,0.108,0.198,0.076,0.302,0.279,0.107,0.187,0.112,0.040,0.048,0.105,0.094,0.177,0.069,0.176,0.061,0.163,0.199,0.234,0.298,0.071,0.099,0.132,0.295,0.134,0.182,0.093,0.459,0.141,0.183,0.235,0.135,0.084,0.289,0.326,0.206,0.044,0.145,0.360,0.152,0.190,0.126,0.154,0.287,0.319,0.275,0.146,0.174
ABBV,0.129,0.000,0.155,0.153,0.142,0.352,0.287,0.136,0.165,0.198,0.276,0.066,0.719,0.267,0.333,0.264,0.299,0.221,0.276,0.407,0.163,0.453,0.198,0.203,0.233,0.222,0.346,0.091,0.259,0.446,0.195,0.164,0.233,0.152,0.304,0.326,0.197,0.293,0.221,0.196,0.132,0.156,0.192,0.261,0.172,0.206,0.138,0.218,0.418,0.277,0.371,0.110,0.160,0.085,0.356,0.209,0.291,0.220,0.477,0.178,0.230,0.323,0.232,0.241,0.369,0.401,0.270,0.143,0.211,0.375,0.182,0.309,0.229,0.299,0.412,0.380,0.506,0.263,0.304
ABT,0.064,0.155,0.000,0.152,0.143,0.301,0.356,0.216,0.139,0.137,0.197,0.130,0.573,0.227,0.258,0.118,0.208,0.164,0.170,0.371,0.089,0.336,0.168,0.233,0.301,0.058,0.158,0.088,0.212,0.341,0.163,0.183,0.260,0.096,0.230,0.324,0.042,0.251,0.103,0.058,0.110,0.156,0.122,0.284,0.079,0.241,0.106,0.292,0.232,0.380,0.449,0.153,0.066,0.196,0.392,0.176,0.132,0.135,0.560,0.185,0.170,0.281,0.236,0.112,0.420,0.275,0.234,0.090,0.237,0.514,0.242,0.173,0.241,0.228,0.428,0.322,0.373,0.237,0.221
ACN,0.134,0.153,0.152,0.000,0.157,0.307,0.356,0.219,0.145,0.260,0.207,0.204,0.785,0.323,0.226,0.135,0.329,0.139,0.195,0.345,0.204,0.470,0.188,0.120,0.291,0.230,0.338,0.124,0.239,0.510,0.172,0.096,0.285,0.269,0.383,0.267,0.220,0.200,0.141,0.184,0.215,0.056,0.118,0.262,0.236,0.156,0.109,0.318,0.283,0.387,0.405,0.171,0.219,0.217,0.398,0.127,0.349,0.170,0.573,0.155,0.264,0.390,0.297,0.253,0.453,0.384,0.230,0.189,0.206,0.480,0.241,0.301,0.201,0.184,0.493,0.346,0.412,0.220,0.310
ADBE,0.077,0.142,0.143,0.157,0.000,0.390,0.122,0.069,0.031,0.141,0.199,0.121,0.734,0.249,0.310,0.220,0.310,0.248,0.391,0.413,0.154,0.444,0.280,0.177,0.129,0.153,0.299,0.076,0.348,0.402,0.033,0.197,0.277,0.204,0.316,0.144,0.202,0.295,0.260,0.160,0.078,0.155,0.208,0.069,0.126,0.255,0.051,0.157,0.242,0.194,0.192,0.051,0.206,0.168,0.161,0.253,0.273,0.261,0.308,0.210,0.282,0.184,0.125,0.143,0.356,0.511,0.319,0.093,0.252,0.270,0.081,0.305,0.208,0.213,0.342,0.488,0.434,0.297,0.249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VALE,0.287,0.412,0.428,0.493,0.342,0.343,0.423,0.407,0.383,0.347,0.299,0.350,0.446,0.449,0.377,0.394,0.271,0.378,0.503,0.246,0.244,0.216,0.312,0.354,0.311,0.308,0.321,0.351,0.402,0.236,0.368,0.357,0.190,0.310,0.320,0.610,0.416,0.440,0.440,0.347,0.268,0.449,0.319,0.351,0.369,0.366,0.385,0.266,0.338,0.347,0.389,0.301,0.365,0.315,0.366,0.392,0.364,0.334,0.405,0.293,0.282,0.290,0.234,0.365,0.055,0.373,0.319,0.249,0.387,0.361,0.383,0.332,0.258,0.380,0.000,0.365,0.314,0.306,0.232
VZ,0.319,0.380,0.322,0.346,0.488,0.314,0.745,0.575,0.510,0.412,0.279,0.380,0.419,0.403,0.239,0.214,0.316,0.205,0.166,0.243,0.229,0.235,0.098,0.320,0.553,0.324,0.211,0.377,0.165,0.420,0.518,0.195,0.417,0.328,0.299,0.813,0.282,0.317,0.213,0.286,0.400,0.255,0.221,0.603,0.365,0.309,0.412,0.390,0.240,0.574,0.663,0.438,0.315,0.373,0.718,0.151,0.246,0.168,0.812,0.187,0.275,0.387,0.390,0.447,0.385,0.086,0.152,0.289,0.252,0.607,0.515,0.194,0.187,0.278,0.365,0.000,0.272,0.185,0.252

In [13]:
dist_matrix_all.sort_values("AAPL")[["AAPL"]].T

,AAPL,CAT,LIN,TMO,LLY,MSFT,ABT,CSCO,MRK,DHR,NVO,IBM,ADBE,AVGO,SAP,ASML,PG,AZN,MCD,NVS,GOOGL,MA,JNJ,HD,COST,KO,AXP,BHP,UNH,ABBV,ORCL,PEP,ACN,QCOM,PM,AMZN,TMUS,WMT,BRK.B,TSM,V,CMCSA,NFLX,XOM,MS,META,PFE,PTRCY,CRM,JPM,TTE,BAC,HSBC,NKE,TM,CVX,BP,NOW,PYPL,BUD,DIS,AMD,GE,AIG,WFC,ISRG,VALE,SHEL,PDD,NVDA,C,INTC,CHL,VZ,T,BABA,TSLA,PLTR,BA
AAPL,0.000,0.040,0.040,0.044,0.048,0.061,0.064,0.067,0.069,0.069,0.071,0.076,0.077,0.081,0.084,0.092,0.093,0.093,0.094,0.099,0.104,0.105,0.107,0.108,0.111,0.112,0.112,0.117,0.126,0.129,0.132,0.134,0.134,0.135,0.141,0.143,0.145,0.146,0.152,0.152,0.154,0.158,0.163,0.174,0.176,0.177,0.182,0.183,0.186,0.187,0.190,0.193,0.198,0.199,0.206,0.210,0.230,0.234,0.235,0.236,0.245,0.257,0.260,0.262,0.275,0.279,0.287,0.289,0.295,0.298,0.301,0.302,0.317,0.319,0.326,0.326,0.360,0.459,0.640


*If you don't want to use the cosine distance to calculate the matrix, you can also use `euclidean` distance:

In [14]:
dist_matrix_all = df_slice.distance(orient="cross-sectional", distance='euclidean', calculations=features)

In [15]:
dist_matrix_all.sort_values("AAPL")[["AAPL"]].T

,AAPL,CAT,LIN,LLY,MSFT,TMO,MRK,CSCO,NVO,IBM,ABT,ADBE,AVGO,SAP,ASML,AZN,NVS,GOOGL,PG,DHR,MCD,HD,MA,BHP,AXP,KO,JNJ,ABBV,ORCL,QCOM,COST,UNH,TMUS,PEP,AMZN,WMT,TSM,PM,ACN,NFLX,CMCSA,BRK.B,XOM,V,META,PFE,TTE,PTRCY,MS,JPM,CRM,HSBC,BAC,TM,CVX,BP,BUD,NKE,NOW,PYPL,GE,AIG,AMD,WFC,DIS,SHEL,VALE,PDD,NVDA,ISRG,C,INTC,T,BABA,VZ,CHL,TSLA,PLTR,BA
AAPL,0.000,0.243,0.256,0.264,0.309,0.318,0.318,0.321,0.322,0.336,0.336,0.337,0.343,0.367,0.367,0.370,0.387,0.392,0.399,0.404,0.404,0.406,0.415,0.416,0.417,0.421,0.427,0.431,0.440,0.442,0.448,0.456,0.461,0.465,0.468,0.469,0.472,0.472,0.488,0.493,0.494,0.498,0.499,0.501,0.506,0.520,0.521,0.521,0.525,0.527,0.541,0.543,0.545,0.547,0.552,0.575,0.585,0.588,0.592,0.598,0.606,0.610,0.614,0.623,0.635,0.642,0.642,0.643,0.647,0.652,0.657,0.659,0.675,0.676,0.676,0.686,0.708,0.797,0.921


### Time Series Distance 

#### Pearson Correlation (fast)

You could also try `spearman` for a non-linear monotonic correlation measure.

In [16]:
dist_matrix_pearson = df_slice.distance(orient="time-series", metric="pearson")

In [17]:
dist_matrix_pearson.sort_values("AAPL")[["AAPL"]].T

,AAPL,MSFT,NOW,NVDA,ABT,CAT,ACN,DHR,PEP,CRM,ADBE,GOOGL,ISRG,PG,COST,TMUS,QCOM,IBM,MA,AMZN,PYPL,LIN,XOM,KO,ORCL,TMO,PFE,MCD,CSCO,BABA,WMT,PTRCY,AIG,NFLX,TSLA,ASML,CVX,V,META,NKE,NVS,BP,VZ,TTE,AMD,AVGO,ABBV,JNJ,PDD,TSM,NVO,LLY,SAP,PM,DIS,AZN,T,BUD,GE,JPM,HD,CMCSA,AXP,TM,WFC,MRK,INTC,BA,BRK.B,UNH,PLTR,BHP,MS,C,CHL,BAC,HSBC,SHEL,VALE
AAPL,0.000,0.560,0.594,0.646,0.648,0.649,0.658,0.660,0.665,0.671,0.678,0.681,0.683,0.686,0.704,0.708,0.713,0.714,0.735,0.738,0.740,0.743,0.744,0.744,0.748,0.749,0.749,0.752,0.753,0.754,0.758,0.759,0.759,0.769,0.770,0.771,0.771,0.773,0.775,0.779,0.780,0.782,0.784,0.789,0.789,0.791,0.801,0.802,0.806,0.809,0.810,0.811,0.811,0.813,0.815,0.825,0.826,0.827,0.831,0.833,0.836,0.838,0.838,0.840,0.840,0.841,0.849,0.854,0.856,0.865,0.870,0.875,0.877,0.883,0.891,0.895,0.900,0.919,0.927


#### Dynamic Time Warping (slow)

In [18]:
dist_matrix_dtw = df_slice.distance(orient="time-series", metric="dtw")

In [23]:
dist_matrix_dtw.sort_values("AAPL")[["AAPL"]].T

,AAPL,GOOGL,META,PG,KO,JNJ,HD,BABA,ORCL,WMT,UNH,AMZN,VZ,TM,BAC,MSFT,JPM,WFC,BRK.B,V,MRK,MA,XOM,ABBV,NVDA,PEP,NVS,TSLA,MCD,TSM,CVX,CSCO,PM,AVGO,CRM,C,MS,LLY,SHEL,ASML,DIS,TMO,IBM,AXP,CMCSA,NVO,ABT,ADBE,PTRCY,COST,PFE,T,ACN,SAP,LIN,TTE,AZN,DHR,CHL,HSBC,BA,GE,QCOM,TMUS,CAT,BP,BUD,BHP,NFLX,ISRG,INTC,AIG,NKE,NOW,AMD,PDD,PYPL,PLTR,VALE
AAPL,0.000,0.003,0.003,0.003,0.003,0.003,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.005,0.005,0.005,0.005,0.007,0.012,0.021,0.022,0.029,0.030,0.035,0.038,0.039,0.042,0.064,0.068,0.074,0.074,0.076,0.076,0.077,0.079,0.079,0.081,0.089,0.090,0.098,0.102,0.104,0.106,0.109,0.111,0.116,0.123,0.129,0.129,0.131,0.131,0.136,0.165,0.177,0.218,0.221,0.222,0.222,0.223,0.224,0.225,0.230,0.235,0.245,0.246,0.251,0.262,0.267,0.268,0.269,0.270,0.275,0.310


#### Various Other Measures

The distance metrics encompass a diverse range of approaches for comparing time series, capturing various aspects of similarity and difference:

1. `dtw`: Dynamic Time Warping, which allows for non-linear alignment of time series.
2. `pearson` and `spearman`: Correlation-based distances, measuring linear and rank-based relationships respectively.
3. `euclidean` and `euclidean_int`: Euclidean distances, with the latter using interpolation for different-length series.
4. `pec`: Power Envelope Correlation, capturing similarities in the energy distribution of signals.
5. `frechet`: Fréchet distance, measuring similarity between curves.
6. `kl_divergence`: Kullback-Leibler divergence, comparing probability distributions.
7. `wasserstein`: Wasserstein distance, also known as Earth Mover's distance.
8. `jaccard`: Jaccard distance, comparing set similarity.
9. `bray_curtis`: Bray-Curtis dissimilarity, often used in ecology.
10. `hausdorff`: Hausdorff distance, measuring how far two subsets of a metric space are from each other.
11. `manhattan`: Manhattan distance, summing the absolute differences.
12. `chi2`: Chi-squared distance, useful for comparing histograms.
13. `hellinger`: Hellinger distance, measuring the similarity between probability distributions.
14. `canberra`: Canberra distance, weighted version of Manhattan distance.
15. `shannon_entropy`: Distance based on Shannon entropy, measuring information content.
16. `sample_entropy` and `approx_entropy`: Complexity measures based on repeating patterns.
17. `jensen_shannon`: Jensen-Shannon divergence, measuring similarity between probability distributions.
18. `renyi_entropy` and `tsallis_entropy`: Generalized entropy measures.
19. `mutual_information`: Distance based on mutual information, measuring mutual dependence.

These metrics capture a wide range of time series characteristics, including shape, statistical properties, complexity, and information content. They allow for comprehensive comparison of time series data, accommodating different aspects of similarity and difference in temporal patterns.

In [20]:
dist_matrix_dtw = df_slice.distance(orient="time-series", metric="tsallis_entropy")

In [21]:
dist_matrix_dtw.sort_values("AAPL")[["AAPL"]].T

,AAPL,GOOGL,META,PG,KO,JNJ,HD,BABA,ORCL,WMT,UNH,AMZN,VZ,TM,BAC,MSFT,JPM,WFC,BRK.B,V,MRK,MA,XOM,ABBV,NVDA,PEP,NVS,TSLA,MCD,TSM,CVX,CSCO,PM,AVGO,CRM,C,MS,LLY,SHEL,ASML,DIS,TMO,IBM,AXP,CMCSA,NVO,ABT,ADBE,PTRCY,COST,PFE,T,ACN,SAP,LIN,TTE,AZN,DHR,CHL,HSBC,BA,GE,QCOM,TMUS,CAT,BP,BUD,BHP,NFLX,ISRG,INTC,AIG,NKE,NOW,AMD,PDD,PYPL,PLTR,VALE
AAPL,0.000,0.003,0.003,0.003,0.003,0.003,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.005,0.005,0.005,0.005,0.007,0.012,0.021,0.022,0.029,0.030,0.035,0.038,0.039,0.042,0.064,0.068,0.074,0.074,0.076,0.076,0.077,0.079,0.079,0.081,0.089,0.090,0.098,0.102,0.104,0.106,0.109,0.111,0.116,0.123,0.129,0.129,0.131,0.131,0.136,0.165,0.177,0.218,0.221,0.222,0.222,0.223,0.224,0.225,0.230,0.235,0.245,0.246,0.251,0.262,0.267,0.268,0.269,0.270,0.275,0.310


### Panel Interactions

The Tucker decomposition is our only panel distance metric, it can be seen as a generalisation of the CP decomposition: it decomposes the tensor into a small core tensor and factor matrices.

In [34]:
dist_matrix_tucker = df_slice.distance(orient="panel")

Estimated rank: [54, 64, 15]


In [35]:
dist_matrix_tucker

,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMZN,ASML,AVGO,AXP,AZN,BA,BABA,BAC,BHP,BP,BRK.B,BUD,C,CAT,CHL,CMCSA,COST,CRM,CSCO,CVX,DHR,DIS,GE,GOOGL,HD,HSBC,IBM,INTC,ISRG,JNJ,JPM,KO,LIN,LLY,MA,MCD,META,MRK,MS,MSFT,NFLX,NKE,NOW,NVDA,NVO,NVS,ORCL,PDD,PEP,PFE,PG,PLTR,PM,PTRCY,PYPL,QCOM,SAP,SHEL,T,TM,TMO,TMUS,TSLA,TSM,TTE,UNH,V,VALE,VZ,WFC,WMT,XOM
AAPL,0.000,1.046,0.992,0.959,0.763,1.068,1.052,0.863,0.988,1.010,1.027,1.055,1.039,0.998,0.909,0.854,0.881,1.118,0.895,0.995,1.132,1.089,1.152,0.929,1.043,0.883,1.049,0.952,1.090,0.934,0.862,1.137,0.820,0.983,1.044,1.124,0.865,1.012,0.899,0.881,0.989,1.134,0.940,1.053,1.081,0.818,0.724,1.061,0.898,0.930,1.083,1.022,0.847,0.976,1.097,0.913,1.150,0.710,1.021,1.030,0.987,0.922,0.733,0.995,0.919,0.950,1.026,1.120,1.027,0.823,0.954,0.961,1.024,1.115,1.123,1.091,1.205,0.785,1.110
ABBV,1.046,0.000,1.050,0.932,0.958,0.974,0.970,1.023,1.032,0.995,0.978,1.032,0.990,1.033,0.999,1.080,0.988,0.990,0.956,0.966,0.978,1.008,1.059,0.952,0.968,0.953,1.077,0.961,0.962,1.039,1.014,0.905,0.857,1.030,0.968,1.065,0.873,0.990,1.016,0.948,0.944,0.906,1.152,1.037,1.013,0.994,1.083,1.011,0.954,1.030,0.966,0.990,0.928,1.058,1.001,1.009,0.936,0.980,0.996,1.033,1.003,1.011,1.105,1.022,1.069,0.979,1.081,1.003,0.910,0.989,0.849,0.945,1.023,1.045,1.005,1.044,1.079,0.970,0.958
ABT,0.992,1.050,0.000,0.924,1.095,0.879,1.039,0.936,0.942,0.949,0.973,1.122,1.044,0.940,1.031,0.865,1.108,0.946,1.023,0.945,1.097,1.003,0.948,1.200,0.971,0.914,0.931,1.051,0.834,0.984,1.155,0.962,0.982,1.021,1.007,0.887,0.580,1.046,0.953,0.872,0.942,0.921,0.788,1.175,0.991,1.071,1.118,0.886,1.045,0.979,0.963,1.001,0.861,1.047,0.993,1.085,0.883,0.992,1.010,1.019,0.992,0.960,0.936,1.012,0.909,0.932,0.915,0.775,0.986,0.984,0.904,1.042,1.047,1.070,1.168,1.241,1.040,1.137,1.087
ACN,0.959,0.932,0.924,0.000,0.524,0.978,0.928,1.023,1.026,1.075,0.932,1.068,1.071,1.017,1.106,0.836,1.029,0.824,1.041,1.094,1.004,0.978,1.082,0.734,0.953,1.064,0.902,0.819,0.902,1.066,0.896,0.707,0.916,1.026,0.999,0.862,1.101,1.065,0.944,0.793,1.016,0.921,0.751,0.964,1.114,0.910,1.025,1.068,1.014,0.977,1.076,1.134,1.150,0.819,0.991,1.173,1.005,1.181,1.008,1.112,0.986,1.203,0.978,1.031,1.128,0.880,0.891,1.064,0.993,1.012,1.086,1.019,0.937,0.847,1.028,1.101,0.990,1.121,1.072
ADBE,0.763,0.958,1.095,0.524,0.000,0.964,0.991,1.060,0.874,1.082,1.094,1.060,0.991,0.848,1.099,0.964,1.041,1.041,0.976,1.141,1.178,0.985,0.906,1.086,1.006,1.084,1.053,0.977,0.986,0.999,0.903,0.907,1.087,1.036,0.938,1.110,1.142,0.964,0.873,0.764,0.976,1.119,1.106,0.868,0.947,1.003,0.963,0.846,1.089,0.906,1.001,0.923,0.878,0.981,1.055,1.020,0.938,1.064,0.993,1.003,1.007,0.971,0.939,1.040,1.010,0.968,1.051,1.195,1.037,1.024,1.092,0.938,0.973,0.952,0.990,1.054,0.896,1.096,1.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VALE,1.123,1.005,1.168,1.028,0.990,0.955,1.003,1.064,0.989,1.010,1.080,1.008,1.005,1.071,0.974,0.813,0.875,0.857,0.924,1.109,0.816,1.006,0.985,1.176,0.955,1.003,1.144,1.006,0.936,1.036,0.991,1.053,0.787,0.957,0.946,1.005,1.022,1.083,1.037,1.002,1.042,1.010,0.954,1.046,0.978,0.926,0.959,0.993,0.963,0.919,0.953,1.044,0.984,1.036,0.953,0.986,0.842,1.153,1.025,0.930,1.014,0.843,0.905,1.023,0.825,1.037,1.073,0.979,1.002,1.022,1.017,1.077,0.870,0.917,0.000,1.003,0.981,1.099,1.073
VZ,1.091,1.044,1.241,1.101,1.054,1.065,0.898,1.063,1.171,1.064,1.114,0.849,1.097,0.945,0.855,0.840,0.967,1.012,1.044,0.828,1.026,0.992,0.760,0.806,1.078,1.008,1.067,0.928,0.944,1.086,1.021,0.927,1.043,0.945,0.885,1.103,1.096,0.932,0.940,1.091,1.091,0.934,1.098,0.948,0.944,1.229,0.904,0.918,0.939,1.011,1.139,0.944,1.118,1.075,1.085,1.042,0.933,0.768,0.986,0.984,1.023,1.092,0.837,0.888,0.956,0.700,0.907,0.932,0.843,1.008,0.790,0.891,1.014,0.949,1.003,0.000,0.971,0.970,1.082

In [36]:
dist_matrix_tucker.sort_values("AAPL")[["AAPL"]].T

,AAPL,PG,MSFT,QCOM,ADBE,WMT,MS,HSBC,TSLA,NVS,BHP,GOOGL,AMZN,JNJ,LIN,BP,CSCO,BUD,NKE,KO,BAC,PEP,SHEL,PYPL,COST,NOW,GE,MCD,T,DHR,TSM,ACN,TTE,ORCL,IBM,PTRCY,ASML,LLY,ABT,SAP,C,BABA,AVGO,JPM,PLTR,NVO,UNH,TM,TMUS,AXP,PM,BA,CRM,INTC,ABBV,CVX,AMD,META,AZN,NFLX,AIG,MRK,NVDA,CHL,DIS,VZ,PDD,XOM,V,BRK.B,TMO,VALE,ISRG,CAT,MA,HD,PFE,CMCSA,WFC
AAPL,0.000,0.710,0.724,0.733,0.763,0.785,0.818,0.820,0.823,0.847,0.854,0.862,0.863,0.865,0.881,0.881,0.883,0.895,0.898,0.899,0.909,0.913,0.919,0.922,0.929,0.930,0.934,0.940,0.950,0.952,0.954,0.959,0.961,0.976,0.983,0.987,0.988,0.989,0.992,0.995,0.995,0.998,1.010,1.012,1.021,1.022,1.024,1.026,1.027,1.027,1.030,1.039,1.043,1.044,1.046,1.049,1.052,1.053,1.055,1.061,1.068,1.081,1.083,1.089,1.090,1.091,1.097,1.110,1.115,1.118,1.120,1.123,1.124,1.132,1.134,1.137,1.150,1.152,1.205


## Date Distance Matrices

Every distance calculation above was done be default accross `on="ticker"` we can also specify `on="date"`.

In [37]:
## All previous functions converted to "date"
dist_matrix_mean = df_slice.distance(on="date")
dist_matrix_cos = df_slice.distance(orient="cross-sectional", on="date", distance='cosine', calculations=features)
dist_matrix_euc = df_slice.distance(orient="cross-sectional", on="date", distance='euclidean', calculations=features)
dist_matrix_pearson = df_slice.distance(orient="time-series", on="date", metric="pearson")
dist_matrix_dtw = df_slice.distance(orient="time-series", on="date", metric="dtw")
dist_matrix_tsent = df_slice.distance(orient="time-series", on="date", metric="tsallis_entropy")
dist_matrix_tucker = df_slice.distance(orient="panel", on="date")

Estimated rank: [54, 64, 15]


In [38]:
dist_matrix_cos

,2020-01-31,2020-02-07,2020-02-14,2020-02-21,2020-02-28,2020-03-06,2020-03-13,2020-03-20,2020-03-27,2020-04-03,2020-04-10,2020-04-17,2020-04-24,2020-05-01,2020-05-08,2020-05-15,2020-05-22,2020-05-29,2020-06-05,2020-06-12,2020-06-19,2020-06-26,2020-07-03,2020-07-10,2020-07-17,2020-07-24,2020-07-31,2020-08-07,2020-08-14,2020-08-21,2020-08-28,2020-09-04,2020-09-11,2020-09-18,2020-09-25,2020-10-02,2020-10-09,2020-10-16,2020-10-23,2020-10-30,2020-11-06,2020-11-13,2020-11-20,2020-11-27,2020-12-04,2020-12-11,2020-12-18,2020-12-25,2021-01-01,2021-01-08,2021-01-15,2021-01-22,2021-01-29,2021-02-05,2021-02-12,2021-02-19,2021-02-26,2021-03-05,2021-03-12,2021-03-19,2021-03-26,2021-04-02,2021-04-09,2021-04-16,2021-04-23,2021-04-30,2021-05-07,2021-05-14,2021-05-21,2021-05-28,2021-06-04,2021-06-11,2021-06-18,2021-06-25,2021-07-02,2021-07-09,2021-07-16,2021-07-23,2021-07-30,2021-08-06,2021-08-13,2021-08-20,2021-08-27,2021-09-03,2021-09-10,2021-09-17,2021-09-24,2021-10-01,2021-10-08,2021-10-15,2021-10-22,2021-10-29,2021-11-05,2021-11-12,2021-11-19,2021-11-26,2021-12-03,2021-12-10,2021-12-17,2021-12-24,2021-12-31,2022-01-07,2022-01-14,2022-01-21,2022-01-28,2022-02-04,2022-02-11,2022-02-18,2022-02-25,2022-03-04,2022-03-11,2022-03-18,2022-03-25,2022-04-01,2022-04-08,2022-04-15,2022-04-22,2022-04-29,2022-05-06,2022-05-13,2022-05-20,2022-05-27,2022-06-03,2022-06-10,2022-06-17,2022-06-24,2022-07-01,2022-07-08,2022-07-15,2022-07-22,2022-07-29,2022-08-05,2022-08-12,2022-08-19,2022-08-26,2022-09-02,2022-09-09,2022-09-16,2022-09-23,2022-09-30,2022-10-07,2022-10-14,2022-10-21,2022-10-28,2022-11-04,2022-11-11,2022-11-18,2022-11-25,2022-12-02,2022-12-09,2022-12-16,2022-12-23,2022-12-30,2023-01-06,2023-01-13,2023-01-20,2023-01-27,2023-02-03,2023-02-10,2023-02-17,2023-02-24,2023-03-03,2023-03-10,2023-03-17,2023-03-24,2023-03-31,2023-04-07,2023-04-14,2023-04-21,2023-04-28,2023-05-05,2023-05-12,2023-05-19,2023-05-26,2023-06-02,2023-06-09,2023-06-16,2023-06-23,2023-06-30,2023-07-07,2023-07-14,2023-07-21,2023-07-28,2023-08-04,2023-08-11,2023-08-18,2023-08-25,2023-09-01,2023-09-08,2023-09-15,2023-09-22,2023-09-29,2023-10-06,2023-10-13,2023-10-20,2023-10-27,2023-11-03,2023-11-10,2023-11-17,2023-11-24,2023-12-01,2023-12-08,2023-12-15,2023-12-22,2023-12-29,2024-01-05,2024-01-12,2024-01-19,2024-01-26,2024-02-02,2024-02-09,2024-02-16,2024-02-23,2024-03-01,2024-03-08,2024-03-15,2024-03-22,2024-03-29,2024-04-05,2024-04-12,2024-04-19,2024-04-26,2024-05-03,2024-05-10,2024-05-17,2024-05-24,2024-05-31,2024-06-07,2024-06-14,2024-06-21,2024-06-28,2024-07-05,2024-07-12,2024-07-19,2024-07-26,2024-08-02,2024-08-09,2024-08-16,2024-08-23,2024-08-30,2024-09-06,2024-09-13,2024-09-20,2024-09-27,2024-10-04,2024-10-11,2024-10-18,2024-10-25,2024-11-01,2024-11-08,2024-11-15,2024-11-22,2024-11-29,2024-12-06,2024-12-13,2024-12-20,2024-12-27,2025-01-03,2025-01-10,2025-01-17,2025-01-24,2025-01-31,2025-02-07,2025-02-14,2025-02-21,2025-02-28,2025-03-07,2025-03-14,2025-03-21,2025-03-28,2025-04-04,2025-04-11,2025-04-18,2025-04-25,2025-05-02
2020-01-31,0.000,0.059,0.111,0.087,0.066,0.133,0.337,0.222,0.163,0.265,0.113,0.150,0.199,0.195,0.142,0.181,0.184,0.177,0.121,0.185,0.136,0.276,0.154,0.221,0.163,0.190,0.162,0.117,0.185,0.196,0.274,0.192,0.162,0.118,0.224,0.147,0.190,0.165,0.167,0.140,0.217,0.163,0.141,0.150,0.152,0.126,0.249,0.159,0.132,0.083,0.224,0.297,0.205,0.148,0.261,0.309,0.214,0.326,0.413,0.319,0.554,0.543,0.384,0.685,0.485,0.380,0.455,0.414,0.310,0.333,0.335,0.199,0.211,0.244,0.307,0.396,0.388,0.385,0.354,0.389,0.431,0.374,0.318,0.321,0.284,0.279,0.385,0.242,0.375,0.236,0.303,0.399,0.310,0.243,0.118,0.332,0.390,0.256,0.218,0.218,0.280,0.268,0.222,0.179,0.215,0.197,0.157,0.099,0.115,0.109,0.110,0.197,0.208,0.153,0.206,0.232,0.137,0.168,0.246,0.188,0.116,0.197,0.193,0.120,0.130,0.259,0.125,0.180,0.270,0.217,0.156,0.271,0.150,0.249,0.218,0.239,0.218,0.189,0.154,0.113,0.244,0.261,0.385,0.269,0.242,0.290,0.260,0.332,0.367,0.314,0.262,0.356,0.308,0.228,0.261,0.201,0.261,0.216,0.216,0.172,0.1

In [39]:
date = dist_matrix_cos.index.max()
dist_matrix_cos.sort_values(date)[[date]].T

,2025-05-02,2025-04-25,2025-04-18,2021-10-15,2025-03-28,2023-08-11,2023-12-29,2024-04-05,2021-09-10,2024-05-03,2020-05-29,2020-06-12,2024-05-10,2020-05-15,2024-03-29,2025-04-11,2021-09-17,2023-06-23,2020-04-17,2024-05-17,2021-08-27,2024-01-05,2023-02-24,2023-06-16,2024-03-15,2021-09-03,2024-03-22,2021-12-24,2023-02-17,2023-02-10,2024-04-19,2021-10-22,2020-02-14,2023-06-09,2021-06-25,2020-06-19,2020-03-20,2023-01-20,2021-09-24,2024-01-12,2020-06-05,2025-03-21,2020-04-03,2024-12-27,2024-04-26,2023-12-08,2020-04-10,2024-01-19,2021-11-19,2023-07-14,2023-05-26,2023-11-03,2024-04-12,2023-06-02,2020-09-04,2020-05-22,2020-05-01,2020-05-08,2021-10-01,2025-03-14,2021-06-18,2021-12-10,2021-06-11,2025-02-21,2023-01-13,2024-05-31,2023-10-06,2021-11-05,2023-02-03,2024-08-30,2024-01-26,2025-04-04,2023-03-03,2021-10-08,2020-07-10,2021-07-09,2023-01-06,2021-07-02,2024-07-05,2023-12-15,2020-02-21,2024-11-08,2023-01-27,2025-03-07,2021-12-31,2024-12-20,2021-05-21,2023-12-22,2023-03-10,2021-07-16,2024-06-21,2023-10-27,2020-07-17,2020-03-27,2023-06-30,2021-06-04,2024-12-13,2022-01-07,2024-11-29,2020-03-06,2024-11-22,2023-03-17,2023-12-01,2023-10-20,2023-08-04,2023-11-17,2023-08-25,2024-10-04,2021-10-29,2024-07-19,2020-03-13,2022-12-16,2021-07-30,2022-12-30,2020-07-31,2021-07-23,2022-01-28,2024-08-02,2023-09-22,2023-05-19,2020-07-24,2020-10-23,2024-08-16,2023-07-07,2024-07-12,2021-12-17,2022-02-04,2023-05-05,2024-10-25,2021-01-01,2021-08-20,2024-11-01,2020-07-03,2023-08-18,2020-04-24,2020-09-11,2023-11-10,2024-10-11,2024-08-23,2021-08-06,2025-01-03,2024-03-08,2020-01-31,2020-02-07,2020-06-26,2025-02-14,2022-12-23,2023-04-14,2024-08-09,2024-02-02,2024-06-07,2021-05-28,2023-11-24,2022-04-01,2024-03-01,2025-01-24,2023-05-12,2024-05-24,2021-12-03,2020-11-06,2024-11-15,2024-12-06,2022-05-20,2022-02-18,2024-06-14,2020-02-28,2023-07-21,2020-10-30,2020-11-20,2022-02-25,2022-02-11,2025-02-28,2022-03-04,2020-10-02,2022-07-29,2021-11-12,2024-06-28,2023-07-28,2023-09-29,2022-07-22,2020-08-14,2021-11-26,2022-04-08,2022-03-25,2020-08-07,2020-09-25,2024-10-18,2022-06-03,2020-12-04,2022-03-18,2024-02-23,2020-08-28,2023-09-08,2020-10-16,2022-03-11,2023-10-13,2024-02-09,2022-06-10,2023-09-15,2020-10-09,2020-12-11,2021-01-08,2025-01-31,2022-09-30,2020-11-27,2024-09-06,2020-11-13,2022-07-01,2021-05-14,2022-06-17,2024-07-26,2024-09-27,2023-09-01,2023-03-31,2020-09-18,2021-08-13,2023-03-24,2020-08-21,2025-01-17,2022-05-27,2022-04-22,2022-08-19,2022-09-23,2021-02-05,2020-12-25,2023-04-07,2022-01-14,2024-09-13,2022-05-13,2023-04-21,2022-04-15,2022-01-21,2024-02-16,2025-02-07,2022-10-14,2022-07-08,2022-09-16,2025-01-10,2022-08-12,2023-04-28,2021-02-26,2020-12-18,2022-12-09,2022-04-29,2022-12-02,2024-09-20,2022-11-04,2021-04-09,2022-10-07,2022-09-09,2022-06-24,2021-05-07,2022-07-15,2021-01-15,2021-01-29,2022-08-26,2022-09-02,2022-05-06,2022-11-18,2021-04-30,2022-10-28,2021-03-19,2021-02-12,2021-03-05,2022-08-05,2022-11-11,2022-11-25,2021-01-22,2021-02-19,2022-10-21,2021-04-23,2021-04-02,2021-03-26,2021-04-16,2021-03-12
2025-05-02,0.000,0.032,0.039,0.041,0.045,0.048,0.054,0.058,0.062,0.064,0.065,0.069,0.070,0.071,0.072,0.074,0.075,0.081,0.081,0.082,0.082,0.082,0.083,0.085,0.085,0.087,0.087,0.089,0.091,0.091,0.095,0.097,0.097,0.098,0.099,0.099,0.099,0.100,0.101,0.101,0.102,0.104,0.104,0.105,0.107,0.110,0.111,0.112,0.112,0.112,0.113,0.116,0.116,0.116,0.118,0.118,0.118,0.120,0.120,0.120,0.122,0.122,0.123,0.124,0.126,0.128,0.129,0.129,0.129,0.132,0.133,0.135,0.136,0.140,0.140,0.140,0.140,0.142,0.144,0.144,0.147,0.148,0.149,0.149,0.150,0.150,0.152,0.152,0.153,0.155,0.155,0.157,0.158,0.160,0.162,0.162,0.163,0.164,0.165,0.165,0.166,0.166,0.166,0.168,0.168,0.169,0.169,0.169,0.170,0.173,0.174,0.175,0.176,0.178,0.180,0.180,0.180,0.181,0.182,0.182,0.183,0.183,0.183,0.184,0.185,0.187,0.189,0.191,0.191,0.192,0.193,0.194,0.194,0.195,0.198,0.198,0.199,0.200,0.200,0.200,0.202,0.203,0.204,0.204,0.204,0.205,0.206,0.207,0.208,0.209,0.209,0.212,0.214,0.214,0.217,0.219,0.219,0.219,0.220,0.224,0.2